# Crime Type Breakdown Bar Chart

Stacked bar chart showing crime composition by Major Crime Indicator (MCI) categories.

## 1. Data Reference

### Source Tables

| Table | Grain | Key Columns |
|-------|-------|-------------|
| `mart_neighbourhood_safety` | neighbourhood × year | assault_count, auto_theft_count, break_enter_count, robbery_count, etc. |

### SQL Query

In [1]:
import os

import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine

# Load .env from project root
load_dotenv("../../.env")

engine = create_engine(os.environ["DATABASE_URL"])

query = """
SELECT
    neighbourhood_name,
    assault_count,
    auto_theft_count,
    break_enter_count,
    robbery_count,
    theft_over_count,
    homicide_count,
    total_incidents,
    crime_rate_per_100k
FROM mart_toronto.mart_neighbourhood_safety
WHERE year = (SELECT MAX(year) FROM mart_toronto.mart_neighbourhood_safety)
ORDER BY total_incidents DESC NULLS LAST
LIMIT 15
"""

df = pd.read_sql(query, engine)
print(f"Loaded top {len(df)} neighbourhoods by crime volume")

Loaded top 15 neighbourhoods by crime volume


### Transformation Steps

1. Select top 15 neighbourhoods by total incidents
2. Melt crime type columns into rows
3. Pass to stacked bar figure factory

In [2]:
df_melted = df.melt(
    id_vars=["neighbourhood_name", "total_incidents"],
    value_vars=[
        "assault_count",
        "auto_theft_count",
        "break_enter_count",
        "robbery_count",
        "theft_over_count",
        "homicide_count",
    ],
    var_name="crime_type",
    value_name="count",
)

# Clean labels
df_melted["crime_type"] = (
    df_melted["crime_type"].str.replace("_count", "").str.replace("_", " ").str.title()
)

data = df_melted.to_dict("records")

### Sample Output

In [3]:
df[
    [
        "neighbourhood_name",
        "assault_count",
        "auto_theft_count",
        "break_enter_count",
        "total_incidents",
    ]
].head(10)

,neighbourhood_name,assault_count,auto_theft_count,break_enter_count,total_incidents
0,St Lawrence-East Bayfront-The Islands,473,11,73,45346
1,Annex,416,45,149,39261
2,West Humber-Clairville,349,494,130,38743
3,Wellington Place,442,37,111,38530
4,Harbourfront-CityPlace,312,20,50,35787
5,Mount Olive-Silverstone-Jamestown,272,95,19,34977
6,York University Heights,428,213,126,34571
7,Banbury-Don Mills,132,72,81,33009
8,Tam O'Shanter-Sullivan,143,67,40,32400
9,Glenfield-Jane Heights,345,72,36,32389


## 2. Data Visualization

### Figure Factory

Uses `create_stacked_bar` from `portfolio_app.figures.toronto.bar_charts`.

In [4]:
import sys

sys.path.insert(0, "../..")

from portfolio_app.figures.toronto.bar_charts import create_stacked_bar

fig = create_stacked_bar(
    data=data,
    x_column="neighbourhood_name",
    value_column="count",
    category_column="crime_type",
    title="Crime Type Breakdown - Top 15 Neighbourhoods",
    color_map={
        "Assault": "#d62728",
        "Auto Theft": "#ff7f0e",
        "Break Enter": "#9467bd",
        "Robbery": "#8c564b",
        "Theft Over": "#e377c2",
        "Homicide": "#1f77b4",
    },
)

fig.show()

### MCI Categories

| Category | Description |
|----------|------------|
| Assault | Physical attacks |
| Auto Theft | Vehicle theft |
| Break & Enter | Burglary |
| Robbery | Theft with force/threat |
| Theft Over | Theft > $5,000 |
| Homicide | Murder/manslaughter |